# Advent of Code 2017, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202017%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [1]:
⎕IO←1
]box on -style=max -trains=tree -fns=on
]rows on -fold=3

done

┌→─────────────────────────────────────┐
│Was ON -style=min -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→────────────────┐
│Was OFF -fold=off│
└─────────────────┘

### Day 1: Inverse Captcha
https://adventofcode.com/2017/day/1

In [5]:
DAY1←⊃⊃⎕NGET'data/2017/01.txt'1

Tack on the first item to the end, as per the problem statement.

In [8]:
DATA←DAY1,DAY1[1]

We use a length-2 windowed reduction using equality to find all location where item n is equal to item n+1 as a binary map. We convert this to position (`⍸`) and pick the corresponding items. As we're still dealing with characters, we need to convert to integers (`⍎¨`) and then just sum them up.

In [12]:
+/⍎¨DATA[⍸2=/DATA] ⍝ Part 1: 1341

1341

For part 2, we create a matrix where column 1 is the input and column 2 is the data rotated by half its length. Then compare the elements of each row to produce the binary map, and the rest is as per part 1.

In [17]:
+/⍎¨DAY1[⍸=/⍉↑(DAY1)(DAY1⊖⍨2÷⍨≢DAY1)] ⍝ Part 2: 1348

1348